In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case0.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (4) – 2 (5) – 0
Route 2: 0 – 3 (6) – 0
Total cost: 31
Number of deliveries: 3
Trucks loads: 9 6


In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case1.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Total cost: 24000
Number of deliveries: 8
Trucks loads: 60 90 60 90 60 90 60 90


In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case2.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Total cost: 80000
Number of deliveries: 16
Trucks loads: 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90


In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case3.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Total cost: 48000
Number of deliveries: 16
Trucks loads: 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90


In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case4.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Total cost: 72000
Number of deliveries: 24
Trucks loads: 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90


In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case5.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Total cost: 159998
Number of deliveries: 32
Trucks loads: 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90


In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case6.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Total cost: 96000
Number of deliveries: 32
Trucks loads: 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90 60 90


In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case7.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Route 33: 0 – 33 (60) – 0
Route 34: 0 – 34 (90) – 0
Route 35: 0 – 35 (60) – 0
Route 36: 0 – 36 (90) – 0
Route 37: 0 – 37 (60) – 0
Route 38: 0 – 38 (90) – 0
Route 3

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case8.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Route 33: 0 – 33 (60) – 0
Route 34: 0 – 34 (90) – 0
Route 35: 0 – 35 (60) – 0
Route 36: 0 – 36 (90) – 0
Route 37: 0 – 37 (60) – 0
Route 38: 0 – 38 (90) – 0
Route 3

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case9.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Route 33: 0 – 33 (60) – 0
Route 34: 0 – 34 (90) – 0
Route 35: 0 – 35 (60) – 0
Route 36: 0 – 36 (90) – 0
Route 37: 0 – 37 (60) – 0
Route 38: 0 – 38 (90) – 0
Route 3

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case10.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Route 33: 0 – 33 (60) – 0
Route 34: 0 – 34 (90) – 0
Route 35: 0 – 35 (60) – 0
Route 36: 0 – 36 (90) – 0
Route 37: 0 – 37 (60) – 0
Route 38: 0 – 38 (90) – 0
Route 3

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case11.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Route 33: 0 – 33 (60) – 0
Route 34: 0 – 34 (90) – 0
Route 35: 0 – 35 (60) – 0
Route 36: 0 – 36 (90) – 0
Route 37: 0 – 37 (60) – 0
Route 38: 0 – 38 (90) – 0
Route 3

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case12.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Route 33: 0 – 33 (60) – 0
Route 34: 0 – 34 (90) – 0
Route 35: 0 – 35 (60) – 0
Route 36: 0 – 36 (90) – 0
Route 37: 0 – 37 (60) – 0
Route 38: 0 – 38 (90) – 0
Route 3

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case13.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Route 33: 0 – 33 (60) – 0
Route 34: 0 – 34 (90) – 0
Route 35: 0 – 35 (60) – 0
Route 36: 0 – 36 (90) – 0
Route 37: 0 – 37 (60) – 0
Route 38: 0 – 38 (90) – 0
Route 3

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case14.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Route 33: 0 – 33 (60) – 0
Route 34: 0 – 34 (90) – 0
Route 35: 0 – 35 (60) – 0
Route 36: 0 – 36 (90) – 0
Route 37: 0 – 37 (60) – 0
Route 38: 0 – 38 (90) – 0
Route 3

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case15.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Route 33: 0 – 33 (60) – 0
Route 34: 0 – 34 (90) – 0
Route 35: 0 – 35 (60) – 0
Route 36: 0 – 36 (90) – 0
Route 37: 0 – 37 (60) – 0
Route 38: 0 – 38 (90) – 0
Route 3

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case16.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Route 33: 0 – 33 (60) – 0
Route 34: 0 – 34 (90) – 0
Route 35: 0 – 35 (60) – 0
Route 36: 0 – 36 (90) – 0
Route 37: 0 – 37 (60) – 0
Route 38: 0 – 38 (90) – 0
Route 3

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case17.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Route 33: 0 – 33 (60) – 0
Route 34: 0 – 34 (90) – 0
Route 35: 0 – 35 (60) – 0
Route 36: 0 – 36 (90) – 0
Route 37: 0 – 37 (60) – 0
Route 38: 0 – 38 (90) – 0
Route 3

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case18.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Route 33: 0 – 33 (60) – 0
Route 34: 0 – 34 (90) – 0
Route 35: 0 – 35 (60) – 0
Route 36: 0 – 36 (90) – 0
Route 37: 0 – 37 (60) – 0
Route 38: 0 – 38 (90) – 0
Route 3

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case19.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Route 33: 0 – 33 (60) – 0
Route 34: 0 – 34 (90) – 0
Route 35: 0 – 35 (60) – 0
Route 36: 0 – 36 (90) – 0
Route 37: 0 – 37 (60) – 0
Route 38: 0 – 38 (90) – 0
Route 3

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case20.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Route 33: 0 – 33 (60) – 0
Route 34: 0 – 34 (90) – 0
Route 35: 0 – 35 (60) – 0
Route 36: 0 – 36 (90) – 0
Route 37: 0 – 37 (60) – 0
Route 38: 0 – 38 (90) – 0
Route 3

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case21.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (60) – 0
Route 2: 0 – 2 (90) – 0
Route 3: 0 – 3 (60) – 0
Route 4: 0 – 4 (90) – 0
Route 5: 0 – 5 (60) – 0
Route 6: 0 – 6 (90) – 0
Route 7: 0 – 7 (60) – 0
Route 8: 0 – 8 (90) – 0
Route 9: 0 – 9 (60) – 0
Route 10: 0 – 10 (90) – 0
Route 11: 0 – 11 (60) – 0
Route 12: 0 – 12 (90) – 0
Route 13: 0 – 13 (60) – 0
Route 14: 0 – 14 (90) – 0
Route 15: 0 – 15 (60) – 0
Route 16: 0 – 16 (90) – 0
Route 17: 0 – 17 (60) – 0
Route 18: 0 – 18 (90) – 0
Route 19: 0 – 19 (60) – 0
Route 20: 0 – 20 (90) – 0
Route 21: 0 – 21 (60) – 0
Route 22: 0 – 22 (90) – 0
Route 23: 0 – 23 (60) – 0
Route 24: 0 – 24 (90) – 0
Route 25: 0 – 25 (60) – 0
Route 26: 0 – 26 (90) – 0
Route 27: 0 – 27 (60) – 0
Route 28: 0 – 28 (90) – 0
Route 29: 0 – 29 (60) – 0
Route 30: 0 – 30 (90) – 0
Route 31: 0 – 31 (60) – 0
Route 32: 0 – 32 (90) – 0
Route 33: 0 – 33 (60) – 0
Route 34: 0 – 34 (90) – 0
Route 35: 0 – 35 (60) – 0
Route 36: 0 – 36 (90) – 0
Route 37: 0 – 37 (60) – 0
Route 38: 0 – 38 (90) – 0
Route 3

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case22.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 2 (700) – 1 (1100) – 3 (800) – 4 (1400) – 0
Route 2: 0 – 10 (600) – 9 (500) – 7 (800) – 5 (2100) – 6 (400) – 8 (100) – 11 (1200) – 0
Route 3: 0 – 13 (1300) – 16 (2100) – 14 (300) – 15 (900) – 12 (1300) – 0
Route 4: 0 – 18 (900) – 17 (1000) – 19 (2500) – 0
Route 5: 0 – 20 (1800) – 21 (700) – 0
Total cost: 485
Number of deliveries: 21
Trucks loads: 4000 5700 5900 4400 2500


In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case23.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 2 (84) – 3 (60) – 1 (125) – 6 (175) – 9 (1100) – 5 (300) – 4 (500) – 8 (150) – 7 (350) – 0
Route 2: 0 – 10 (4100) – 11 (225) – 0
Route 3: 0 – 12 (300) – 13 (250) – 16 (100) – 15 (150) – 14 (500) – 17 (250) – 22 (75) – 20 (500) – 19 (600) – 18 (120) – 21 (175) – 0
Total cost: 672
Number of deliveries: 22
Trucks loads: 2844 4325 3020


In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case24.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 3 (125) – 6 (150) – 1 (300) – 4 (100) – 5 (200) – 2 (3100) – 7 (150) – 0
Route 2: 0 – 18 (150) – 14 (150) – 8 (450) – 11 (950) – 12 (125) – 9 (300) – 17 (100) – 13 (150) – 16 (150) – 15 (550) – 10 (100) – 19 (400) – 20 (300) – 0
Route 3: 0 – 23 (300) – 21 (1500) – 22 (100) – 24 (500) – 25 (800) – 27 (100) – 28 (150) – 26 (300) – 0
Route 4: 0 – 29 (1000) – 0
Total cost: 776
Number of deliveries: 29
Trucks loads: 4125 3875 3750 1000


In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case25.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 4 (1200) – 5 (40) – 6 (80) – 7 (2000) – 9 (600) – 8 (900) – 10 (750) – 1 (700) – 2 (400) – 3 (400) – 0
Route 2: 0 – 12 (150) – 11 (1500) – 13 (250) – 14 (1600) – 15 (450) – 17 (550) – 16 (700) – 20 (400) – 21 (300) – 19 (200) – 18 (650) – 0
Route 3: 0 – 22 (1300) – 23 (700) – 24 (750) – 25 (1400) – 0
Route 4: 0 – 26 (4000) – 27 (600) – 28 (1000) – 29 (500) – 0
Route 5: 0 – 32 (1100) – 31 (1700) – 30 (2500) – 0
Total cost: 1133
Number of deliveries: 32
Trucks loads: 7070 6750 4150 6100 5300


In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case26.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (7) – 3 (16) – 2 (30) – 5 (21) – 4 (9) – 6 (15) – 7 (19) – 8 (23) – 9 (11) – 10 (5) – 0
Route 2: 0 – 15 (10) – 17 (3) – 13 (23) – 14 (21) – 12 (29) – 16 (15) – 11 (19) – 0
Route 3: 0 – 19 (9) – 21 (8) – 20 (28) – 22 (8) – 26 (7) – 23 (16) – 24 (10) – 25 (28) – 18 (41) – 0
Route 4: 0 – 27 (15) – 31 (11) – 28 (14) – 32 (12) – 37 (9) – 33 (23) – 30 (19) – 34 (26) – 29 (6) – 35 (17) – 36 (6) – 0
Route 5: 0 – 38 (15) – 39 (14) – 45 (10) – 44 (16) – 42 (13) – 40 (7) – 41 (27) – 47 (25) – 43 (11) – 48 (17) – 46 (5) – 0
Route 6: 0 – 49 (18) – 50 (10) – 0
Total cost: 1042
Number of deliveries: 50
Trucks loads: 156 120 155 158 160 28


In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case27.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 6 (19) – 3 (11) – 1 (18) – 2 (26) – 5 (21) – 4 (30) – 7 (15) – 0
Route 2: 0 – 12 (16) – 9 (29) – 10 (26) – 11 (37) – 8 (16) – 13 (12) – 0
Route 3: 0 – 15 (8) – 20 (22) – 16 (19) – 18 (13) – 17 (20) – 14 (31) – 19 (15) – 0
Route 4: 0 – 21 (28) – 22 (12) – 23 (6) – 24 (27) – 25 (14) – 26 (18) – 27 (17) – 0
Route 5: 0 – 29 (13) – 30 (22) – 28 (29) – 32 (28) – 31 (25) – 0
Route 6: 0 – 33 (27) – 36 (12) – 37 (14) – 34 (19) – 35 (10) – 38 (24) – 39 (16) – 0
Route 7: 0 – 45 (21) – 42 (11) – 41 (15) – 43 (18) – 44 (17) – 40 (33) – 0
Route 8: 0 – 52 (19) – 48 (20) – 47 (19) – 49 (5) – 50 (22) – 51 (12) – 46 (27) – 0
Route 9: 0 – 53 (22) – 58 (21) – 55 (7) – 56 (26) – 57 (14) – 54 (16) – 59 (24) – 0
Route 10: 0 – 60 (13) – 61 (15) – 64 (28) – 62 (18) – 63 (11) – 65 (9) – 66 (37) – 0
Route 11: 0 – 75 (20) – 68 (10) – 74 (10) – 70 (11) – 71 (3) – 69 (8) – 73 (6) – 72 (1) – 67 (30) – 0
Total cost: 1804
Number of deliveries: 75
Trucks loads: 140 136 128 122 117 12

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case28.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (10) – 9 (16) – 3 (13) – 2 (7) – 14 (20) – 5 (26) – 8 (9) – 7 (5) – 11 (12) – 10 (16) – 12 (19) – 4 (19) – 13 (23) – 6 (3) – 0
Route 2: 0 – 15 (8) – 16 (19) – 17 (2) – 18 (12) – 19 (17) – 20 (9) – 21 (11) – 22 (18) – 23 (29) – 25 (6) – 24 (3) – 29 (9) – 26 (17) – 28 (16) – 27 (16) – 0
Route 3: 0 – 31 (27) – 30 (21) – 33 (11) – 34 (14) – 35 (8) – 32 (23) – 36 (5) – 37 (8) – 38 (16) – 43 (7) – 42 (5) – 41 (5) – 39 (31) – 40 (9) – 0
Route 4: 0 – 44 (18) – 45 (16) – 46 (1) – 49 (30) – 47 (27) – 48 (36) – 52 (9) – 51 (10) – 50 (13) – 54 (18) – 55 (2) – 56 (6) – 53 (14) – 0
Route 5: 0 – 58 (18) – 57 (7) – 59 (28) – 61 (13) – 60 (3) – 62 (19) – 64 (9) – 63 (10) – 66 (25) – 65 (20) – 67 (25) – 0
Route 6: 0 – 69 (6) – 70 (5) – 71 (15) – 78 (3) – 79 (23) – 73 (9) – 74 (8) – 75 (18) – 72 (25) – 80 (6) – 68 (36) – 77 (14) – 76 (13) – 0
Route 7: 0 – 81 (26) – 90 (3) – 88 (9) – 82 (16) – 83 (11) – 84 (7) – 86 (35) – 91 (1) – 85 (41) – 92 (2) – 87 (26) – 89 (15) 

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case29.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 3 (11) – 1 (18) – 2 (26) – 4 (30) – 0
Route 2: 0 – 6 (19) – 5 (21) – 8 (16) – 7 (15) – 9 (29) – 0
Route 3: 0 – 13 (12) – 11 (37) – 10 (26) – 12 (16) – 0
Route 4: 0 – 14 (31) – 15 (8) – 16 (19) – 18 (13) – 17 (20) – 0
Route 5: 0 – 19 (15) – 20 (22) – 21 (28) – 22 (12) – 23 (6) – 0
Route 6: 0 – 24 (27) – 25 (14) – 26 (18) – 27 (17) – 0
Route 7: 0 – 31 (25) – 28 (29) – 30 (22) – 29 (13) – 0
Route 8: 0 – 32 (28) – 33 (27) – 36 (12) – 34 (19) – 35 (10) – 0
Route 9: 0 – 37 (14) – 38 (24) – 39 (16) – 40 (33) – 0
Route 10: 0 – 44 (17) – 43 (18) – 41 (15) – 42 (11) – 45 (21) – 0
Route 11: 0 – 46 (27) – 48 (20) – 47 (19) – 49 (5) – 50 (22) – 0
Route 12: 0 – 51 (12) – 55 (7) – 53 (22) – 54 (16) – 52 (19) – 0
Route 13: 0 – 56 (26) – 60 (13) – 57 (14) – 59 (24) – 58 (21) – 0
Route 14: 0 – 62 (18) – 61 (15) – 64 (28) – 63 (11) – 65 (9) – 0
Route 15: 0 – 66 (37) – 72 (1) – 68 (10) – 69 (8) – 71 (3) – 70 (11) – 67 (30) – 0
Route 16: 0 – 73 (6) – 74 (10) – 75 (20) – 

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case30.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 1 (10) – 9 (16) – 3 (13) – 4 (19) – 2 (7) – 6 (3) – 5 (26) – 8 (9) – 7 (5) – 0
Route 2: 0 – 11 (12) – 10 (16) – 12 (19) – 15 (8) – 14 (20) – 13 (23) – 0
Route 3: 0 – 16 (19) – 17 (2) – 18 (12) – 19 (17) – 20 (9) – 21 (11) – 22 (18) – 0
Route 4: 0 – 23 (29) – 25 (6) – 24 (3) – 29 (9) – 26 (17) – 28 (16) – 27 (16) – 0
Route 5: 0 – 31 (27) – 36 (5) – 32 (23) – 30 (21) – 35 (8) – 34 (14) – 33 (11) – 0
Route 6: 0 – 37 (8) – 42 (5) – 40 (9) – 39 (31) – 41 (5) – 43 (7) – 38 (16) – 44 (18) – 0
Route 7: 0 – 45 (16) – 46 (1) – 49 (30) – 47 (27) – 48 (36) – 0
Route 8: 0 – 53 (14) – 52 (9) – 51 (10) – 50 (13) – 54 (18) – 55 (2) – 56 (6) – 57 (7) – 58 (18) – 0
Route 9: 0 – 59 (28) – 61 (13) – 60 (3) – 62 (19) – 64 (9) – 63 (10) – 65 (20) – 0
Route 10: 0 – 71 (15) – 66 (25) – 70 (5) – 69 (6) – 68 (36) – 67 (25) – 0
Route 11: 0 – 73 (9) – 74 (8) – 75 (18) – 72 (25) – 78 (3) – 77 (14) – 76 (13) – 0
Route 12: 0 – 80 (6) – 79 (23) – 81 (26) – 82 (16) – 83 (11) – 84 (7

In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case31.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 6 (19) – 3 (11) – 1 (18) – 2 (26) – 5 (21) – 4 (30) – 8 (16) – 7 (15) – 0
Route 2: 0 – 9 (29) – 10 (26) – 11 (37) – 14 (31) – 13 (12) – 15 (8) – 16 (19) – 12 (16) – 0
Route 3: 0 – 17 (20) – 26 (18) – 19 (15) – 20 (22) – 21 (28) – 22 (12) – 23 (6) – 24 (27) – 18 (13) – 25 (14) – 0
Route 4: 0 – 34 (19) – 27 (17) – 29 (13) – 30 (22) – 31 (25) – 32 (28) – 33 (27) – 28 (29) – 0
Route 5: 0 – 36 (12) – 37 (14) – 35 (10) – 38 (24) – 39 (16) – 40 (33) – 44 (17) – 41 (15) – 42 (11) – 43 (18) – 0
Route 6: 0 – 52 (19) – 45 (21) – 48 (20) – 47 (19) – 49 (5) – 50 (22) – 51 (12) – 53 (22) – 46 (27) – 0
Route 7: 0 – 56 (26) – 55 (7) – 58 (21) – 59 (24) – 54 (16) – 57 (14) – 60 (13) – 61 (15) – 62 (18) – 63 (11) – 0
Route 8: 0 – 72 (1) – 65 (9) – 66 (37) – 67 (30) – 74 (10) – 70 (11) – 71 (3) – 69 (8) – 64 (28) – 73 (6) – 68 (10) – 75 (20) – 0
Total cost: 1586
Number of deliveries: 75
Trucks loads: 156 178 175 180 170 167 165 173


In [ ]:
import numpy as np

# Fonction pour charger les données depuis un fichier d'instance
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Première ligne : Nombre de clients et capacité des véhicules
    num_clients, vehicle_capacity = map(int, lines[0].split())

    # Deuxième ligne : Demandes des clients
    demands = list(map(int, lines[1].split()))

    # Coordonnées des nœuds (dépôt + clients)
    coordinates = []
    for line in lines[2:]:
        if line.strip():  # Ignorer les lignes vides
            parts = line.split()
            if len(parts) == 2:  # Vérifie que la ligne contient exactement deux valeurs
                x, y = map(int, parts)
                coordinates.append((x, y))
            else:
                print(f"Ligne ignorée ou mal formatée : {line}")

    return num_clients, vehicle_capacity, demands, coordinates

# Fonction pour calculer la matrice des distances euclidiennes
def calculate_distance_matrix(coordinates):
    num_nodes = len(coordinates)
    distance_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                distance = np.sqrt((coordinates[j][0] - coordinates[i][0])**2 +
                                   (coordinates[j][1] - coordinates[i][1])**2)
                distance_matrix[i][j] = int(distance + 0.5)  # Arrondi comme spécifié

    return distance_matrix

# Fonction pour générer une solution initiale simple
def generate_initial_solution(num_clients, vehicle_capacity, demands):
    routes = []
    current_route = []
    current_capacity = 0

    for client in range(1, num_clients + 1):
        if current_capacity + demands[client - 1] > vehicle_capacity:
            routes.append([0] + current_route + [0])
            current_route = []
            current_capacity = 0
        current_route.append(client)
        current_capacity += demands[client - 1]

    if current_route:
        routes.append([0] + current_route + [0])

    return routes

# Fonction pour calculer le coût total d'une solution
def calculate_cost(routes, distance_matrix):
    total_cost = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_cost += distance_matrix[route[i]][route[i + 1]]
    return total_cost

# Génère les voisins d'une solution
def generate_neighbors(solution):
    neighbors = []
    for route in solution:
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route) - 1):
                neighbor = [r[:] for r in solution]  # Copie profonde
                neighbor_route = neighbor[solution.index(route)]
                neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
                neighbors.append(neighbor)
    return neighbors

# Métaheuristique : Recherche Tabou
def tabu_search(initial_solution, distance_matrix, tabu_tenure=5, max_iterations=100):
    current_solution = initial_solution
    current_cost = calculate_cost(current_solution, distance_matrix)
    best_solution = current_solution
    best_cost = current_cost

    tabu_list = []

    for _ in range(max_iterations):
        neighbors = generate_neighbors(current_solution)
        neighbor_costs = [calculate_cost(neighbor, distance_matrix) for neighbor in neighbors]

        # Trouver le meilleur voisin qui n'est pas dans la liste tabou
        best_neighbor = None
        best_neighbor_cost = float('inf')
        for neighbor, cost in zip(neighbors, neighbor_costs):
            if neighbor not in tabu_list and cost < best_neighbor_cost:
                best_neighbor = neighbor
                best_neighbor_cost = cost

        if best_neighbor is None:
            break  # Aucun voisin valide trouvé

        # Mise à jour de la solution courante
        current_solution = best_neighbor
        current_cost = best_neighbor_cost

        # Mise à jour de la meilleure solution si nécessaire
        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        # Mise à jour de la liste tabou
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)  # Supprime le plus ancien élément

    return best_solution, best_cost

# Fonction pour afficher les résultats formatés
def display_solution(routes, distance_matrix, demands):
    total_cost = calculate_cost(routes, distance_matrix)
    total_deliveries = sum(len(route) - 2 for route in routes)  # Exclure le dépôt (0) au début et à la fin
    truck_loads = [sum(demands[client - 1] for client in route if client != 0) for route in routes]

    print("Solution:")
    for i, route in enumerate(routes):
        route_display = " – ".join(f"{client} ({demands[client - 1]})" if client != 0 else "0" for client in route)
        print(f"Route {i + 1}: {route_display}")
    print(f"Total cost: {total_cost}")
    print(f"Number of deliveries: {total_deliveries}")
    print(f"Trucks loads: {' '.join(map(str, truck_loads))}")

# Fonction principale pour résoudre une instance donnée
def solve_instance(file_path):
    # Charger les données
    num_clients, vehicle_capacity, demands, coordinates = load_instance(file_path)

    # Calculer la matrice des distances
    distance_matrix = calculate_distance_matrix(coordinates)

    # Générer une solution initiale
    initial_solution = generate_initial_solution(num_clients, vehicle_capacity, demands)

    # Appliquer la recherche Tabou
    best_solution, best_cost = tabu_search(initial_solution, distance_matrix)

    return best_solution, best_cost

# Exemple d'utilisation
if __name__ == "__main__":
    # Chemin vers le fichier d'instance
    instance_file = "/content/Case32.txt"  # Remplacez par le chemin de votre fichier

    # Résoudre l'instance
    solution, cost = solve_instance(instance_file)

    # Charger les données pour afficher les résultats
    _, _, demands, coordinates = load_instance(instance_file)
    distance_matrix = calculate_distance_matrix(coordinates)

    # Afficher les résultats formatés
    print("\nRésultats :")
    display_solution(solution, distance_matrix, demands)



Résultats :
Solution:
Route 1: 0 – 6 (19) – 3 (11) – 1 (18) – 2 (26) – 5 (21) – 4 (30) – 8 (16) – 7 (15) – 10 (26) – 9 (29) – 0
Route 2: 0 – 20 (22) – 15 (8) – 13 (12) – 19 (15) – 14 (31) – 11 (37) – 12 (16) – 18 (13) – 16 (19) – 17 (20) – 0
Route 3: 0 – 28 (29) – 22 (12) – 23 (6) – 24 (27) – 25 (14) – 31 (25) – 27 (17) – 29 (13) – 21 (28) – 30 (22) – 26 (18) – 0
Route 4: 0 – 40 (33) – 33 (27) – 36 (12) – 37 (14) – 34 (19) – 35 (10) – 38 (24) – 39 (16) – 32 (28) – 41 (15) – 42 (11) – 0
Route 5: 0 – 46 (27) – 45 (21) – 48 (20) – 47 (19) – 51 (12) – 43 (18) – 49 (5) – 50 (22) – 44 (17) – 53 (22) – 54 (16) – 52 (19) – 0
Route 6: 0 – 55 (7) – 58 (21) – 65 (9) – 59 (24) – 57 (14) – 60 (13) – 61 (15) – 62 (18) – 64 (28) – 56 (26) – 63 (11) – 0
Route 7: 0 – 67 (30) – 75 (20) – 68 (10) – 74 (10) – 70 (11) – 71 (3) – 69 (8) – 73 (6) – 72 (1) – 66 (37) – 0
Total cost: 1548
Number of deliveries: 75
Trucks loads: 211 193 211 209 218 186 136
